# Projectable set

In [ ]:
from src.v1.symbolic import Var
from src.v2.transformations import flatten_component
from src.v2.execution import Component
from src.v4.problem import Subproblem, intersection
from src.v4.functionalsets import ResidualSet
from src.v4.functionalsets import EliminationSet
from src.v4.functionalsets import EliminationKeepSet
from src.v4.functionalsets import Functional
from src.v4.functionalsets import DefaultResidualSolver
from src.v4.functionalsets import FeedForwardSolver
from src.v4.functionalsets import FunctionalComp
import numpy as np

In [2]:
x_1, x_2, x_3, x_4, x_5 = Var('x_1'), Var('x_2'), Var('x_3'), Var('x_4'), Var('x_5')
# residuals:
#x_1**3 + x_2**3 + x_3**3-3, x_1*x_2*x_3*x_5 - 1
eq1 = Component.fromsympy(x_2**2 + x_3**2-2, arg_mapping=True) 
eq2 = Component.fromsympy(x_2*x_3*x_5 - 1, arg_mapping=True)
S1 = ResidualSet([eq1, eq2])

## Transform to functional

In [3]:
projected = (x_5,)
F1 = S1.project(projected)
solver = F1.solver = DefaultResidualSolver(F1) # uses the residual representation

# Alternative 1
F_eq2_x2 = ResidualSet([eq2]).project((x_3, x_5))
F_eq1_x3 = EliminationSet([eq1], eliminate=F_eq2_x2).project()
F_elim = EliminationKeepSet([eq1], eliminate=F_eq2_x2).project((x_5,))
# Alternative 2
# Sometimes this will be equivalent to DefaultResidualSolver, but not always
# F1.solver = CoupledSolver(F_eq1.solve, F_eq2.solve) 

In [4]:
F_elim.solve({x_5: 2})

{x_2: 0.36602540378443865, x_3: 1.3660254037844386}

In [5]:
F_eq2_x2.solve({x_3:1, x_5:2})
# TODO : make this work
out1 = F_eq1_x3.solve({x_5: 2})

In [6]:
out2 = F_eq2_x2.solve({**out1, x_5:2})
{**out1, **out2}

{x_3: 0.366025403784429, x_2: 1.3660254037844746}

In [7]:
F1.independent

(x_2, x_3)

In [8]:
y1 = {x_1:1, x_4:1, x_5:2}
rescalc = solver.generate_residual_vector(y1) 
rescalc(np.array([0, 0]))

array([-2, -1])

In [9]:
out = solver.solve(y1)

In [10]:
out

{x_2: 0.3660254037843778, x_3: 1.3660254037845079}

In [11]:
solver.generate_residual(y1)(out)

array([ 1.44773082e-13, -1.15574217e-13])

## And a raw functional

In [12]:
def black_box(x3, x2,x5):
    x1 = (x3**2+x2**2+1)**0.5
    x4 = 10+x2+x3-x1-x5
    return x1, x4
eq3 = Component(black_box, (x_3,x_2,x_5), (x_1,x_4), arg_mapping=True)
F2 = FunctionalComp(eq3)
#F2.add_component(residual)

In [13]:
y2 = {x_3: 1, x_2: 1, x_5:2}
out = F2.solve(y2)

In [14]:
out

{x_1: 1.7320508075688772, x_4: 8.267949192431123}

### What if we turned the black box into a residual and solve the residual?

In [15]:
residual = flatten_component(eq3)
projectable = ResidualSet([residual])

In [16]:
residual.evaldict({**out, **y2})

[DeviceArray([0., 0.], dtype=float64, weak_type=True)]

In [17]:
F2worse_solve = projectable.project((x_3,x_2,x_5))

In [18]:
F2worse_solve.solve({**y2, x_1:0.5, x_4:1.37})

{x_1: 1.7320508075688772, x_4: 8.267949192431123}

# Merging sets

In [19]:
# Merging components:
S3 = S1.merge(F2.projectable)
F3 = S3.project((x_5,))

In [20]:
y3 = {x_5:2}

In [21]:
out = F3.solve(y3)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 2 has 2 dimension(s)

In [22]:
out

{x_3: 0.36602540371954434,
 x_2: 1.3660254037891002,
 x_1: 1.7320508074965615,
 x_4: 8.000000000012083}

### Validate results

In [23]:
F1.solve({**out, **y3}), F2.solve({**out, **y3}), 

({x_3: 0.36602540378444665, x_2: 1.3660254037844275},
 {x_1: 1.7320508075588399, x_4: 7.999999999949804})

### Feedforward because there is no coupling

In [22]:
F3.solver = FeedForwardSolver([F1, F2])

In [23]:
F3.solve(y3)

{x_2: 0.36602540378442877,
 x_3: 1.3660254037844637,
 x_1: 1.732050807568895,
 x_4: 7.999999999999998}

### Coupled solvers if there was coupling (but has to work in feed forward mode to)

In [24]:
S3.components

[(('x_2', 'x_3'), None, (None,), 'x_2**2 + x_3**2 - 2'),
 (('x_5', 'x_2', 'x_3'), None, (None,), 'x_2*x_3*x_5 - 1'),
 (('x_3', 'x_2', 'x_5', 'x_1', 'x_4'), None, (None,), 'None')]